In [1]:
%pwd

'/home/invernici/TETYS/pipeline/src/python/notebooks'

In [2]:
%cd ../../../../dashboard/backend/resources/projects/

/home/invernici/TETYS/dashboard/backend/resources/projects


In [3]:
import duckdb
import pandas as pd

In [4]:
GROUP_NO=1

## Read the parquet file

In [5]:
cmd = f"""
CREATE TABLE metadata_clean AS SELECT * FROM read_parquet('./group{GROUP_NO}/metadata.parquet')
"""

duckdb.query(cmd)

```python
Index(['Unnamed: 0', 'prism_doi', 'prism_issn', 'dc_identifier', 'eid',
       'dc_title', 'abstract', 'prism_aggregation_type', 'prism_url',
       'prism_volume', 'openaccess', 'language', 'dc_publisher', 'pubmed_id',
       'author_keywords', 'subject_area', 'authors', 'prism_cover_date',
       'abstract2', 'pub_year', 'pub_month', 'pub_day', 'full_text',
       '__index_level_0__'],
      dtype='object')

```

In [6]:
duckdb.query("""
SELECT 
    COUNT(*) AS total_rows,
    COUNT(DISTINCT prism_doi) AS unique_rows,
    COUNT(prism_doi) AS non_null_rows
FROM 
    metadata_clean;""")

┌────────────┬─────────────┬───────────────┐
│ total_rows │ unique_rows │ non_null_rows │
│   int64    │    int64    │     int64     │
├────────────┼─────────────┼───────────────┤
│      33769 │       33769 │         33769 │
└────────────┴─────────────┴───────────────┘

## Add authors_name and authors_name_shortened columns

In [7]:
duckdb.query("ALTER TABLE metadata_clean ADD COLUMN authors_names VARCHAR;")

In [8]:
duckdb.query("""
UPDATE metadata_clean AS mc
SET authors_names = (
    SELECT 
        CASE
            WHEN len((new.authors::JSON)->'$[*].*[0]')> 1
            THEN list_reduce(((new.authors::JSON)->>'$[*].*[0]'), (x,y) -> concat_ws(', ', x, y))::VARCHAR
            ELSE coalesce((authors::JSON)->>'$[*].*[0]', '')::VARCHAR
        END
    FROM metadata_clean AS new
    WHERE mc.prism_doi = new.prism_doi
    )
""")

In [9]:
duckdb.query("ALTER TABLE metadata_clean ADD COLUMN authors_names_shortened VARCHAR;")

In [10]:
# duckdb.query("""
# UPDATE metadata_clean AS mc
# SET authors_names_shortened = (
#     SELECT 
#         CASE
#             WHEN len((new.authors::JSON)->'$[*].*[0]')> 3
#             THEN list_reduce(((new.authors::JSON)->>'$[*].*[0]')[0:3], (x,y) -> concat_ws(', ', x, y))::VARCHAR || ' et al.'
#             WHEN len((new.authors::JSON)->'$[*].*[0]')> 1 AND len((new.authors::JSON)->'$[*].*[0]') <= 3
#             THEN list_reduce(((new.authors::JSON)->>'$[*].*[0]'), (x,y) -> concat_ws(', ', x, y))::VARCHAR
#             ELSE coalesce(((authors::JSON)->>'$[*].*[0]'), '')::VARCHAR
#         END
#     FROM metadata_clean AS new
#     WHERE mc.prism_doi = new.prism_doi
#     )
# """)

In [11]:
duckdb.query("""
UPDATE metadata_clean AS mc
SET authors_names_shortened = (
    SELECT 
        CASE
            WHEN len((new.authors::JSON)->'$[*].*[0]')> 3
            THEN list_reduce(((new.authors::JSON)->>'$[*].*[0]')[0:3], (x,y) -> concat_ws(', ', x, y))::VARCHAR || ' et al.'
            WHEN len((new.authors::JSON)->'$[*].*[0]')> 0 AND len((new.authors::JSON)->'$[*].*[0]') <= 3
            THEN list_reduce(((new.authors::JSON)->>'$[*].*[0]'), (x,y) -> concat_ws(', ', x, y))::VARCHAR
            ELSE ''::VARCHAR
        END
    FROM metadata_clean AS new
    WHERE mc.prism_doi = new.prism_doi
    )
""")

### Test

In [12]:
df = duckdb.query("""
SELECT (new.authors::JSON)->>'$[*].*[0]' as au, list_reduce((new.authors::JSON)->>'$[*].*[0]', (x,y) -> concat_ws(', ', x, y)) as au_str, authors_names, authors_names_shortened
    FROM metadata_clean AS new
   WHERE len(au) > 1
""").df()
#df.rename(columns = {df.columns[0] : "author_names"},inplace=True)

In [13]:
df.head()

,au,au_str,authors_names,authors_names_shortened
0,"[Paranhos J., Hasenclever L., Perin F.S.]","Paranhos J., Hasenclever L., Perin F.S.","Paranhos J., Hasenclever L., Perin F.S.","Paranhos J., Hasenclever L., Perin F.S."
1,"[Racaj M., Agushi A., Racaj A.]","Racaj M., Agushi A., Racaj A.","Racaj M., Agushi A., Racaj A.","Racaj M., Agushi A., Racaj A."
2,"[Ruiz L.A., King L.F.E., Castillo T.I.L., Garc...","Ruiz L.A., King L.F.E., Castillo T.I.L., Garci...","Ruiz L.A., King L.F.E., Castillo T.I.L., Garci...","Ruiz L.A., King L.F.E., Castillo T.I.L. et al."
3,"[Gaumer G., Crown W.H., Kates J., Luan Y., Har...","Gaumer G., Crown W.H., Kates J., Luan Y., Hari...","Gaumer G., Crown W.H., Kates J., Luan Y., Hari...","Gaumer G., Crown W.H., Kates J. et al."
4,"[Jain R., Gaur A., Suravajhala R., Chauhan U.,...","Jain R., Gaur A., Suravajhala R., Chauhan U., ...","Jain R., Gaur A., Suravajhala R., Chauhan U., ...","Jain R., Gaur A., Suravajhala R. et al."


In [14]:
df[ df.au_str != df.authors_names]

,au,au_str,authors_names,authors_names_shortened


In [15]:
df = duckdb.query("""
SELECT (new.authors::JSON)->>'$[*].*[0]' as au, list_reduce((new.authors::JSON)->>'$[*].*[0]', (x,y) -> concat_ws(', ', x, y)) as au_str, authors_names, authors_names_shortened
    FROM metadata_clean AS new
   WHERE len(au) = 1
""").df()
#df.rename(columns = {df.columns[0] : "author_names"},inplace=True)

In [16]:
df.head()

,au,au_str,authors_names,authors_names_shortened
0,[Bell C.],Bell C.,[Bell C.],Bell C.
1,[Vedeshin L.A.],Vedeshin L.A.,[Vedeshin L.A.],Vedeshin L.A.
2,[Wu M.],Wu M.,[Wu M.],Wu M.
3,[Kim K.O.],Kim K.O.,[Kim K.O.],Kim K.O.
4,[Nordlund C.],Nordlund C.,[Nordlund C.],Nordlund C.


## Add the doi_url column

In [17]:
duckdb.query("ALTER TABLE metadata_clean ADD COLUMN doi_url VARCHAR;")

In [18]:
duckdb.query("""
UPDATE metadata_clean AS mc
SET doi_url = (
    SELECT concat('https://doi.org/', new.prism_doi)
    FROM metadata_clean AS new
    WHERE mc.prism_doi = new.prism_doi
)
""")

### Test

In [19]:
duckdb.query("SELECT prism_doi, doi_url FROM metadata_clean LIMIT 10")

┌─────────────────────────────────┬─────────────────────────────────────────────────┐
│            prism_doi            │                     doi_url                     │
│             varchar             │                     varchar                     │
├─────────────────────────────────┼─────────────────────────────────────────────────┤
│ 10.1007/978-3-031-38894-1       │ https://doi.org/10.1007/978-3-031-38894-1       │
│ 10.1017/jme.2023.118            │ https://doi.org/10.1017/jme.2023.118            │
│ 10.47305/JLIA2393516r           │ https://doi.org/10.47305/JLIA2393516r           │
│ 10.33423/jhetp.v23i18.6631      │ https://doi.org/10.33423/jhetp.v23i18.6631      │
│ 10.1136/bmjopen-2022-070221     │ https://doi.org/10.1136/bmjopen-2022-070221     │
│ 10.1016/j.scitotenv.2023.167098 │ https://doi.org/10.1016/j.scitotenv.2023.167098 │
│ 10.3233/ATDE230999              │ https://doi.org/10.3233/ATDE230999              │
│ 10.1016/j.atmosenv.2023.120137  │ https://doi.org/10

## Add the date_ref column

In [20]:
duckdb.query("ALTER TABLE metadata_clean ADD COLUMN date_ref VARCHAR;")

In [21]:
duckdb.query("""
UPDATE metadata_clean AS mc
SET date_ref = (
    SELECT strftime(prism_cover_date, '%Y-%m-%d')
    FROM metadata_clean AS new
    WHERE mc.prism_doi = new.prism_doi
)
""")

### Test

In [22]:
duckdb.query("SELECT prism_cover_date, strftime(prism_cover_date, '%Y-%m-%d'), date_ref FROM metadata_clean LIMIT 10")

┌─────────────────────┬────────────────────────────────────────┬────────────┐
│  prism_cover_date   │ strftime(prism_cover_date, '%Y-%m-%d') │  date_ref  │
│      timestamp      │                varchar                 │  varchar   │
├─────────────────────┼────────────────────────────────────────┼────────────┤
│ 2023-12-30 00:00:00 │ 2023-12-30                             │ 2023-12-30 │
│ 2023-12-29 00:00:00 │ 2023-12-29                             │ 2023-12-29 │
│ 2023-12-26 00:00:00 │ 2023-12-26                             │ 2023-12-26 │
│ 2023-12-22 00:00:00 │ 2023-12-22                             │ 2023-12-22 │
│ 2023-12-21 00:00:00 │ 2023-12-21                             │ 2023-12-21 │
│ 2023-12-20 00:00:00 │ 2023-12-20                             │ 2023-12-20 │
│ 2023-12-15 00:00:00 │ 2023-12-15                             │ 2023-12-15 │
│ 2023-12-15 00:00:00 │ 2023-12-15                             │ 2023-12-15 │
│ 2023-12-15 00:00:00 │ 2023-12-15                             │

## Write final result

In [23]:
duckdb.query(f"COPY metadata_clean TO './group{GROUP_NO}/metadata_prepared.parquet' (FORMAT 'parquet');")